Import libaries you use at the start of the notebook in their own cell.

In [26]:
import unicodecsv
from datetime import datetime as dt

Read in the CSVs and store as lists of dictionaries.

In [27]:
def read_csv(filename):
    with open(filename, 'rb') as f:
        reader = unicodecsv.DictReader(f)
        return list(reader)

enrollments = read_csv('enrollments.csv')
daily_engagement = read_csv('daily-engagement.csv')
project_submissions = read_csv('project-submissions.csv')
    
print enrollments[0]
print daily_engagement[0]
print project_submissions[0]

{u'status': u'canceled', u'is_udacity': u'True', u'is_canceled': u'True', u'join_date': u'2014-11-10', u'account_key': u'448', u'cancel_date': u'2015-01-14', u'days_to_cancel': u'65'}
{u'lessons_completed': u'0.0', u'num_courses_visited': u'1.0', u'total_minutes_visited': u'11.6793745', u'projects_completed': u'0.0', u'acct': u'0', u'utc_date': u'2015-01-09'}
{u'lesson_key': u'3176718735', u'processing_state': u'EVALUATED', u'account_key': u'256', u'assigned_rating': u'UNGRADED', u'completion_date': u'2015-01-16', u'creation_date': u'2015-01-14'}


Fix some of the data types. Everything is stored as strings. Where appropriate, convert:
* string dates to datetime objects
* string numbers to type int
* string Boolean values to type Bool
* `daily_engagement`'s 'acct' key name to 'account_key'

In [28]:
def parse_date(date):
    if date == "":
        return None
    else: return dt.strptime(date, "%Y-%m-%d")

def parse_maybe_int(i):
    if i == "":
        return None
    else:
        return int(i)
    
for enrollment in enrollments:
    enrollment['cancel_date'] = parse_date(enrollment['cancel_date'])
    enrollment['join_date'] = parse_date(enrollment['join_date'])
    enrollment['days_to_cancel'] = parse_maybe_int(enrollment['days_to_cancel'])
    enrollment['is_canceled'] = enrollment['is_canceled'] == 'True'
    enrollment['is_udacity'] = enrollment['is_udacity'] == 'True'

enrollments[0]

{u'account_key': u'448',
 u'cancel_date': datetime.datetime(2015, 1, 14, 0, 0),
 u'days_to_cancel': 65,
 u'is_canceled': True,
 u'is_udacity': True,
 u'join_date': datetime.datetime(2014, 11, 10, 0, 0),
 u'status': u'canceled'}

In [29]:
for engagement in daily_engagement:
    engagement['lessons_completed'] = int(float(engagement['lessons_completed']))
    engagement['num_courses_visited'] = int(float(engagement['num_courses_visited']))
    engagement['projects_completed'] = int(float(engagement['projects_completed']))
    engagement['total_minutes_visited'] = int(float(engagement['total_minutes_visited']))
    engagement['utc_date'] = parse_date(engagement['utc_date'])
    engagement['account_key'] = engagement['acct']
    del[engagement['acct']]
    
daily_engagement[0]

{'account_key': u'0',
 u'lessons_completed': 0,
 u'num_courses_visited': 1,
 u'projects_completed': 0,
 u'total_minutes_visited': 11,
 u'utc_date': datetime.datetime(2015, 1, 9, 0, 0)}

In [30]:
for submission in project_submissions:
    submission['creation_date'] = parse_date(submission['creation_date'])
    submission['completion_date'] = parse_date(submission['completion_date'])
    
project_submissions[0]

{u'account_key': u'256',
 u'assigned_rating': u'UNGRADED',
 u'completion_date': datetime.datetime(2015, 1, 16, 0, 0),
 u'creation_date': datetime.datetime(2015, 1, 14, 0, 0),
 u'lesson_key': u'3176718735',
 u'processing_state': u'EVALUATED'}

Identify the number of student records for each file and the unique students in each file.

In [37]:
def get_unique_stu(data):
    unique_stu = set()
    for data_pt in data:
        unique_stu.add(data_pt['account_key'])
    return unique_stu

unique_enrollments = get_unique_stu(enrollments)
unique_engagement_stu = get_unique_stu(daily_engagement)
unique_project_submitters = get_unique_stu(project_submissions)

print "Unique enrollments:", len(unique_enrollments)
print "Unique engagement students:", len(unique_engagement_stu)
print "Unique project submitters:",len(unique_project_submitters)

Unique enrollments: 1302
Unique engagement students: 1237
Unique project submitters: 743


What is an example student present in `enrollments` data and missing in `unique_engagement_stu` data?

In [35]:
for enrollment in enrollments:
    stu = enrollment['account_key']
    if stu not in unique_engagement_stu:
        print enrollment
        break

{u'status': u'canceled', u'is_udacity': False, u'is_canceled': True, u'join_date': datetime.datetime(2014, 11, 12, 0, 0), u'account_key': u'1219', u'cancel_date': datetime.datetime(2014, 11, 12, 0, 0), u'days_to_cancel': 0}


This is a Udacity dummy account. How many other students are missing from the engagement data?

In [38]:
num_prob_stu = 0
for enrollment in enrollments:
    stu = enrollment['account_key']
    if stu not in unique_engagement_stu and enrollment['join_date'] != enrollment['cancel_date']:
        num_prob_stu += 1
        print enrollment
        
num_prob_stu

{u'status': u'canceled', u'is_udacity': True, u'is_canceled': True, u'join_date': datetime.datetime(2015, 1, 10, 0, 0), u'account_key': u'1304', u'cancel_date': datetime.datetime(2015, 3, 10, 0, 0), u'days_to_cancel': 59}
{u'status': u'canceled', u'is_udacity': True, u'is_canceled': True, u'join_date': datetime.datetime(2015, 3, 10, 0, 0), u'account_key': u'1304', u'cancel_date': datetime.datetime(2015, 6, 17, 0, 0), u'days_to_cancel': 99}
{u'status': u'current', u'is_udacity': True, u'is_canceled': False, u'join_date': datetime.datetime(2015, 2, 25, 0, 0), u'account_key': u'1101', u'cancel_date': None, u'days_to_cancel': None}


3

All three are Udacity test accounts, not real students. How many test accounts exist?

In [41]:
udacity_test_accounts = set()
for enrollment in enrollments:
    if enrollment['is_udacity']:
        udacity_test_accounts.add(enrollment['account_key'])

len(udacity_test_accounts)

6

Remove the test accounts.

In [54]:
def remove_udacity_accounts(data):
    non_udacity_data = []
    for datum in data:
        if datum['account_key'] not in udacity_test_accounts:
            non_udacity_data.append(datum)
    return non_udacity_data

non_udacity_enrollments = remove_udacity_accounts(enrollments)
non_udacity_engagements = remove_udacity_accounts(daily_engagement)
non_udacity_submissions = remove_udacity_accounts(project_submissions)

print len(non_udacity_enrollments)
print len(non_udacity_engagements)
print len(non_udacity_submissions)
non_udacity_engagements[0]

1622
135656
3634


{'account_key': u'0',
 u'lessons_completed': 0,
 u'num_courses_visited': 1,
 u'projects_completed': 0,
 u'total_minutes_visited': 11,
 u'utc_date': datetime.datetime(2015, 1, 9, 0, 0)}

Create a dictionary of paid students.

In [51]:
paid_students = dict()
for enrollment in non_udacity_enrollments:
    if not enrollment['is_canceled'] or enrollment['days_to_cancel'] > 7: # alt: enrollment['days_to_cancel] == None
        if enrollment['account_key'] not in paid_students or \
        enrollment['join_date'] > paid_students[enrollment['account_key']]:
            paid_students[enrollment['account_key']] = enrollment['join_date'] # dict[key] = value
        
len(paid_students)

995

Remove move free trial cancellations.

In [55]:
def remove_free_trial_cancellations(data):
    ndata = []
    for datum in data:
        if datum['account_key'] in paid_students:
            ndata.append(datum)
    return ndata

paid_enrollments = remove_free_trial_cancellations(non_udacity_enrollments)
paid_engagement = remove_free_trial_cancellations(non_udacity_engagements)
paid_submissions = remove_free_trial_cancellations(non_udacity_submissions)

print len(paid_enrollments)
print len(paid_engagement)
print len(paid_submissions)

1293
134549
3618


Collect engagement data for the first week in a list.

In [56]:
def within_one_week(join_date, engagement_date):
    time_delta = engagement_date - join_date
    return time_delta.days < 7

paid_engagement_in_first_wk = list()
for engagement in paid_engagement:
    if within_one_week(paid_students[engagement['account_key']], engagement['utc_date']):
        paid_engagement_in_first_wk.append(engagement)

len(paid_engagement_in_first_wk)

21508